This notebook is created for exercise purposes. I used the open source data provided by the Istanbul Metropolitan Municipality (IMM). The data consists of the ID, time, latitude, longitude, depth, magnitude type, magnitude size, and event location name of each earthquake that occurred in 2019.

What you will learn from this short notebook is:
1. how to download json files
2. how to convert dictionary data to pandas dataframe
3. how to use the foluim library to create a map and place markers on it.

Lastly, there are a number of datasets available on the IMM's website. If you're interested, please visit [IMM](https://data.ibb.gov.tr/en/).

So, let's begin!      
Download the necessary packages

In [ ]:
import pandas as pd
import xlrd
import folium
import requests

Read the json file from url

In [ ]:
df=requests.get("https://data.ibb.gov.tr/datastore/odata3.0/32904f60-8091-4dc3-b527-351dca6c1c22?$format=json").json()
df=df['value']

Transform dictionary data into pandas dataframe

In [ ]:
# define the dataframe columns
column_names = ['ID', 'Time', 'Latitude', 'Longitude','Depth/km','MagType','Magnitude','EventLocationName'] 

# instantiate the dataframe
earthquack = pd.DataFrame(columns=column_names)

In [ ]:
# load dataframe with data
for data in df:
    event_id = data['#EventID'] 
    event_time = data['Time']
    event_lat = data['Latitude']
    event_lon = data['Longitude']
    depth = data['Depth/km']
    magtype = data['MagType']
    magnitude = data['Magnitude']
    locname = data['EventLocationName']
    
    earthquack = earthquack.append({'ID': event_id,
                                    'Time': event_time,
                                    'Latitude': event_lat,
                                    'Longitude': event_lon,
                                    'Depth/km': depth,
                                    'MagType': magtype,
                                    'Magnitude': magnitude,
                                    'EventLocationName': locname}, ignore_index=True)

Let's check to see if we get what we want

In [ ]:
earthquack.head()

I did a google search to find the coordinations, but there are also python libraries (e.g, _geopy_ library) that do the same thing. I just found googling more convenient for this little task.

In [ ]:
# Latitude and Longitude of Istanbul
latitude=41.015137
longitude=28.979530

Let's write a function that returns a color according to the earthquake's depth.
We will use the function in _CircleMarker_.

In [ ]:
def color_producer(rad):
    if rad < 10:
        return 'green'
    elif 10 <= rad < 20:
        return 'orange'
    else:
        return 'red'

Finally, create a map of Istanbul using the values of latitude and longitude.

In [ ]:
map_ist = folium.Map(location=[latitude, longitude], zoom_start=6)

# add markers to map
for lat, lng, location, rad in zip(earthquack['Latitude'], earthquack['Longitude'], earthquack['EventLocationName'],earthquack['Depth/km']):
    label = '{}'.format(location)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=rad*0.6,
        popup=label,
        fill=True,
        weight=0, # border line weight
        fill_color=color_producer(rad),
        fill_opacity=0.7,
        parse_html=False).add_to(map_ist)  
    
map_ist

![](map.png)